In [ ]:
%env JAVA_HOME=/home/tmpdeatheye/.sdkman/candidates/java/current

In [ ]:
from delta import *
from delta.tables import *

In [ ]:
conf = {
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.endpoint": "http://10.1.1.2:9000",
    "spark.hadoop.fs.s3a.access.key": "username",
    "spark.hadoop.fs.s3a.secret.key": "password",
    "spark.hadoop.fs.s3a.path.style.access": "true",
    "spark.sql.catalogImplementation": "hive",
    "spark.hive.metastore.uris": "thrift://10.1.1.7:9083",
    "spark.hive.metastore.schema.verification": "false",
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    "spark.sql.debug.maxToStringFields": "1024"
}

packages = [
    "org.apache.spark:spark-avro_2.12:3.5.1",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "io.delta:delta-spark_2.12:3.1.0"
]

spark = configure_spark_with_delta_pip(
    spark_session_builder=SparkSession.builder.config(map=conf),
    extra_packages=packages
).getOrCreate()

In [ ]:
spark.sql("""
    CREATE SCHEMA billing LOCATION 's3a://lakehouse/bronze/billing/'
""")

In [ ]:
spark.sql("""
    CREATE TABLE billing.invoice (
        order_id BIGINT,
        created_at TIMESTAMP,
        description STRING,
        date DATE
    )
    USING DELTA
    PARTITIONED BY (date)
    LOCATION 's3a://lakehouse/bronze/billing/main/raw/invoice/'
""")

In [ ]:
spark.sql("""
    SELECT * FROM billing.invoice
""").show(truncate=False)